In [ ]:
# Config and imports

import pypsa
import logging
import atlite
import time
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import plotly
from datetime import datetime
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
from lib.optimize import create_and_store_optimize
from lib.network import create_and_store_network
from lib.costs import create_and_store_costs
from lib.cutout import create_and_store_cutout
from lib.availability import create_and_store_availability
from lib.demand import create_and_store_demand
logging.basicConfig(level=logging.INFO)


# Configuration specified in /prototypes/configs
CONFIG_NAME = "default"
# In the config file, we have a secnarios configuration that is used to create files per scenario. In this notebook, we use the following scenario
SCENARIO = "weather=2011,costs=2020,demand=2025,demand-target=0.03,load-target=10,network-nuclear=on,network-h2=on,network-biogas=none,network-onwind-limit=None,network-offwind-limit=None,geography=14"
# Check the config file, or print all possible options with following code:
# %run scenarios.py "list" "$CONFIG_NAME"
%run scenarios.py "validate" "$CONFIG_NAME" "$SCENARIO"

with open(f"../data/result/{SCENARIO}/config.json", "r") as f:
    CONFIG = json.load(f)


In [ ]:
# Setting variables based on config
DATA_PATH=CONFIG["scenario"]["data-path"]
WEATHER_START=CONFIG["scenario"]["weather_start"]
WEATHER_END=CONFIG["scenario"]["weather_end"]
LAN_CODE = CONFIG["scenario"]["geography_lan_code"]
KOM_CODE = CONFIG["scenario"]["geography_kom_code"]

## Wind turbines
WIND_TURBINE = CONFIG["onwind_turbine"]
WIND_TURBINE_OFFSHORE = CONFIG["offwind_turbine"]

## Cost assumptions
ASSUMPTIONS = pd.read_pickle(f"../{DATA_PATH}/costs.pkl")

## Cutout / selection
CUTOUT = atlite.Cutout(f"../{DATA_PATH}/cutout.nc")
SELECTION = gpd.read_file(f"../{DATA_PATH}/selection.shp")
EEZ = gpd.read_file(f"../{DATA_PATH}/eez.shp")
INDEX = pd.to_datetime(pd.read_csv(f"../{DATA_PATH}/time_index.csv")["0"])

# Cutout / availablity
AVAIL_SOLAR = xr.open_dataarray(f"../{DATA_PATH}/avail_solar.nc")
AVAIL_ONWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_onwind.nc")
AVAIL_OFFWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_offwind.nc")

AVAIL_CAPACITY_SOLAR = xr.open_dataarray(f"../{DATA_PATH}/avail_capacity_solar.nc")
AVAIL_CAPACITY_ONWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_capacity_onwind.nc")
AVAIL_CAPACITY_OFFWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_capacity_offwind.nc")

DEMAND = pd.read_csv(f"../{DATA_PATH}/demand.csv")

NETWORK = pypsa.Network()
NETWORK.import_from_netcdf(f"../{DATA_PATH}/network.nc")

STATISTICS = pd.read_pickle(f"../{DATA_PATH}/statistics.pkl")

In [ ]:
# Some weather availability / capacity graphs

fig = plt.figure(figsize=(18, 10))
gs = GridSpec(3, 3, figure=fig)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])

# Solar avail
AVAIL_SOLAR.sel(dim_0=0).plot(cmap="Reds", ax=ax1)
SELECTION.plot(ax=ax1, edgecolor="k", color="None")
CUTOUT.grid.plot(ax=ax1, color="None", edgecolor="grey", ls=":")
ax1.set_title("SOLAR AVAIL")

# Wind (onshore) avail
AVAIL_ONWIND.sel(dim_0=0).plot(cmap="Greens", ax=ax2)
SELECTION.plot(ax=ax2, edgecolor="k", color="None")
CUTOUT.grid.plot(ax=ax2, color="None", edgecolor="grey", ls=":")
ax2.set_title("WIND ONSHORE AVAIL")

# Wind (offshore) avail
AVAIL_OFFWIND.sel(dim_0=0).plot(cmap="Blues", ax=ax3)
SELECTION.plot(ax=ax3, edgecolor="k", color="None")
CUTOUT.grid.plot(ax=ax3, color="None", edgecolor="grey", ls=":")
ax3.set_title("WIND OFFSHORE AVAIL")

# Capacity factors
AVAIL_CAPACITY_SOLAR.plot(color="Red", ax=ax4)
AVAIL_CAPACITY_ONWIND.plot(color="Green", ax=ax5)
AVAIL_CAPACITY_OFFWIND.plot(color="Blue", ax=ax6)

In [ ]:
# Plot network

minx, miny, maxx, maxy = SELECTION.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

bus_colors = ["blue", "gold", "aquamarine", "dodgerblue", "indianred", "magenta"]
fig, ax = plt.subplots(subplot_kw={"projection":ccrs.EqualEarth()})

# Create a custom legend for the bus names
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                            markerfacecolor=color, markersize=10) for bus, color in zip(NETWORK.buses.index, bus_colors)]

# Add the legend to the plot
ax.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)


NETWORK.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=[0.02, 0.02, 0.02, 0.02, 0.02, 0.02], bus_colors=bus_colors)


In [ ]:
## OVERRIDING VARIABLES?
#CONFIG["demand-target"] = 0.05
# And then, to update any step, just uncomment and run
#create_and_store_costs(CONFIG)
#create_and_store_cutout(CONFIG)
#create_and_store_availability(CONFIG)
#create_and_store_demand(CONFIG)
#create_and_store_network(CONFIG)
#create_and_store_optimize(CONFIG)

STATISTICS